In [ ]:
class SimpleDataset(Dataset):
    def __init__(self, num_samples=20, seq_length=20, pattern_strength=1.0):
        """
        Create a simple dataset where the class is determined by a pattern in the sequence.
        Higher pattern_strength makes the pattern more obvious (easier to learn).
        """
        self.seq_length = seq_length
        self.data = []
        self.labels = []
        
        for i in range(num_samples):
            # Generate a random sequence
            seq = np.random.randn(seq_length)
            
            # Create a pattern: if the first value is positive, class = 1, else 0
            # Add some additional signal in the first few elements to make it stronger
            if random.random() > 0.5:
                # Class 1 pattern
                seq[0] += pattern_strength * 2  # Make first value likely positive
                label = 1
            else:
                # Class 0 pattern
                seq[0] -= pattern_strength * 2  # Make first value likely negative
                label = 0
                
            self.data.append(seq)
            self.labels.append(label)
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.data[idx]), torch.tensor(self.labels[idx])

In [ ]:
# class RNNEncoderClassifier(nn.Module):
#     def __init__(self, n_predict, n_features, n_hidden, n_layers, dropout):
#         super().__init__()
#         self.rnn = nn.RNN(input_size=n_features,
#                           hidden_size=n_hidden,
#                           num_layers=n_layers,
#                           nonlinearity="relu",
#                           dropout=dropout,
#                           bidirectional=False,
#                           batch_first=True)
#         self.fc = nn.Linear(n_hidden, n_predict)
#         self.sigmoid = nn.Sigmoid()
#         
#     def forward(self, x):
#         # Make sure x has the right shape [batch, seq_len, features]
#         if x.dim() == 2:
#             x = x.unsqueeze(-1)
#         
#         output, hn = self.rnn(x)
#         x = self.fc(hn[-1])
#         return self.sigmoid(x)

In [ ]:
def main2(config, use_wandb=False):
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Initialize wandb if needed
    if use_wandb:
        try:
            wandb.init(project="model-debugging", 
                       name=config["run_name"],
                       config=config)
        except:
            print("WandB initialization skipped")
            use_wandb = False
    
    # Create tiny dataset for overfitting test
    train_dataset = SimpleDataset(num_samples=200, seq_length=config["n_past"], pattern_strength=2.0)
    val_dataset = SimpleDataset(num_samples=50, seq_length=config["n_past"], pattern_strength=2.0)
    test_dataset = SimpleDataset(num_samples=50, seq_length=config["n_past"], pattern_strength=2.0)
    
    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)
    
    # Create model
    model = RNNEncoderClassifier(n_predict=config["n_predict"],
                                n_features=config["n_features"],
                                n_hidden=config["n_hidden"],
                                n_layers=config["n_layers"],
                                dropout=config["dropout"])
    
    
    # Set up loss function and metrics
    loss_fn = nn.BCELoss()
    calculate_metrics = partial(evaluate_classification_metrics, threshold=config["threshold"])
    
    # Create optimizer
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
    # Create trainer
    trainer = Trainer(model=model,
                     device=device,
                     optimizer=optimizer,
                     loss_fn=loss_fn,
                     train_loader=train_dataloader,
                     val_loader=val_dataloader,
                     calculate_metrics=calculate_metrics,
                     n_epochs=config["n_epochs"],
                     run_name=config["run_name"],
                     patience=config["patience"])
    
    # Train model
    best_model = trainer.train()
    
    # Evaluate on test set
    evaluate_test(best_model, test_dataloader, device, loss_fn, calculate_metrics)
    
    if use_wandb:
        try:
            wandb.finish()
        except:
            pass
    
    return best_model

In [ ]:
def get_config():
    return {
        "run_name": "rnn_encoder_overfit_test",
        "notes": "Testing overfitting capability with tiny dataset",
        
        "is_classification": True,
        "n_features": 1,
        "n_past": 20,
        "n_predict": 1,
        "n_stride": 1,
        
        "model": "rnn_encoder",
        "n_hidden": 32,
        "n_layers": 2,
        "dropout": 0.0,  # Set to 0 for overfitting test
        
        "n_epochs": 200,  # Increase epochs to allow for overfitting
        "batch_size": 15,  # Small batch size for overfitting
        "threshold": 0.5,
        "learning_rate": 0.001,
        "patience": 50,  # Increase patience for overfitting test
    }


In [ ]:
if __name__ == "__main__":
    # Set seeds for reproducibility
    seed = 42
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    # Get configuration
    config = get_config()
    
    # Run with or without wandb
    use_wandb = True  # Set to False to skip wandb logging
    main2(config, use_wandb)